In [ ]:
from flask import Flask, request, jsonify, send_file
import boto3
import pymongo
import os
import time
from dotenv import load_dotenv

app = Flask(__name__)

# Load environment variables
load_dotenv()

# AWS Credentials
AWS_ACCESS_KEY = os.getenv('AWS_ACCESS_KEY')
AWS_SECRET_KEY = os.getenv('AWS_SECRET_KEY')
AWS_REGION = os.getenv('AWS_REGION')

# MongoDB Connection URI
MONGO_URI = os.getenv('MONGO_URI')

# SES Email
EMAIL_SENDER = os.getenv('EMAIL_SENDER')

def create_ec2_instance():
    ec2_resource = boto3.resource('ec2', region_name=AWS_REGION,
                                  aws_access_key_id=AWS_ACCESS_KEY,
                                  aws_secret_access_key=AWS_SECRET_KEY)
    new_instance = ec2_resource.create_instances(
        ImageId='ami-0abcdef1234567890',
        MinCount=1,
        MaxCount=1,
        InstanceType='t2.micro'
    )
    return {'message': f'New EC2 instance started: {new_instance[0].id}'}

def create_rhel_gui_instance():
    ec2_resource = boto3.resource('ec2', region_name=AWS_REGION,
                                  aws_access_key_id=AWS_ACCESS_KEY,
                                  aws_secret_access_key=AWS_SECRET_KEY)
    new_instance = ec2_resource.create_instances(
        ImageId='ami-0abcdef1234567890',
        MinCount=1,
        MaxCount=1,
        InstanceType='t2.micro'
    )
    return {'message': f'New RHEL GUI instance started: {new_instance[0].id}'}

def get_cloud_logs(log_group, log_stream):
    logs_client = boto3.client('logs', region_name=AWS_REGION,
                               aws_access_key_id=AWS_ACCESS_KEY,
                               aws_secret_access_key=AWS_SECRET_KEY)
    log_events = logs_client.get_log_events(
        logGroupName=log_group,
        logStreamName=log_stream,
        limit=10
    )
    return log_events

def transcribe_audio_to_text(bucket, audio_file):
    transcribe_client = boto3.client('transcribe', region_name=AWS_REGION,
                                     aws_access_key_id=AWS_ACCESS_KEY,
                                     aws_secret_access_key=AWS_SECRET_KEY)
    
    job_identifier = 'transcription-job-' + str(int(time.time()))
    audio_uri = f's3://{bucket}/{os.path.basename(audio_file)}'
    
    transcribe_client.start_transcription_job(
        TranscriptionJobName=job_identifier,
        Media={'MediaFileUri': audio_uri},
        MediaFormat='mp3',
        LanguageCode='en-US'
    )
    
    # Polling the job status until it's complete
    while True:
        job_status = transcribe_client.get_transcription_job(TranscriptionJobName=job_identifier)
        status = job_status['TranscriptionJob']['TranscriptionJobStatus']
        if status in ['COMPLETED', 'FAILED']:
            break
        time.sleep(15)
    
    if status == 'COMPLETED':
        transcript_url = job_status['TranscriptionJob']['Transcript']['TranscriptFileUri']
        return {'message': f'Transcription successful. Download the transcript from {transcript_url}'}
    else:
        return {'error': 'Transcription failed'}

def connect_to_mongo():
    try:
        mongo_client = pymongo.MongoClient(MONGO_URI)
        database_list = mongo_client.list_database_names()
        return {'message': f'Successfully connected to MongoDB. Databases available: {database_list}'}
    except pymongo.errors.ConnectionError as connection_error:
        return {'error': f'MongoDB Connection Error: {connection_error}'}
    except Exception as general_error:
        return {'error': f'Error occurred: {general_error}'}

def upload_file_to_s3(bucket, file_path):
    s3_client = boto3.client('s3', region_name=AWS_REGION,
                             aws_access_key_id=AWS_ACCESS_KEY,
                             aws_secret_access_key=AWS_SECRET_KEY)
    file_name = os.path.basename(file_path)
    s3_client.upload_file(file_path, bucket, file_name)
    return {'message': f'File uploaded to S3 bucket {bucket} as {file_name}'}

def execute_lambda_s3_ses(bucket, key):
    s3_client = boto3.client('s3', region_name=AWS_REGION,
                             aws_access_key_id=AWS_ACCESS_KEY,
                             aws_secret_access_key=AWS_SECRET_KEY)
    ses_client = boto3.client('ses', region_name=AWS_REGION,
                              aws_access_key_id=AWS_ACCESS_KEY,
                              aws_secret_access_key=AWS_SECRET_KEY)
    
    s3_object = s3_client.get_object(Bucket=bucket, Key=key)
    email_list = s3_object['Body'].read().decode('utf-8').splitlines()
    
    email_subject = 'Automated Email from AWS Lambda'
    email_body = 'This is a test email generated by AWS Lambda using SES.'
    
    for email_address in email_list:
        if email_address:
            try:
                ses_client.send_email(
                    Source=EMAIL_SENDER,
                    Destination={'ToAddresses': [email_address]},
                    Message={
                        'Subject': {'Data': email_subject},
                        'Body': {'Text': {'Data': email_body}}
                    }
                )
            except Exception as email_error:
                print(f'Failed to send email to {email_address}: {email_error}')
    
    return {'message': 'All emails were sent successfully'}

@app.route('/')
def homepage():
    return send_file('index.html')

@app.route('/aws_operations', methods=['POST'])
def handle_aws_operations():
    try:
        operation = request.form.get('operation')
        
        if operation == 'launch_ec2_instance':
            result = create_ec2_instance()
        elif operation == 'launch_rhel_gui_instance':
            result = create_rhel_gui_instance()
        elif operation == 'access_cloud_logs':
            log_group = request.form.get('log_group_name')
            log_stream = request.form.get('log_stream_name')
            result = get_cloud_logs(log_group, log_stream)
        elif operation == 'audio_to_text_event':
            bucket = request.form.get('bucket_name')
            audio_file = request.files['audio_file']
            audio_path = f'/tmp/{audio_file.filename}'
            audio_file.save(audio_path)
            result = transcribe_audio_to_text(bucket, audio_path)
        elif operation == 'connect_python_to_mongodb':
            result = connect_to_mongo()
        elif operation == 'upload_to_s3':
            bucket = request.form.get('bucket_name')
            file = request.files['file']
            file_path = f'/tmp/{file.filename}'
            file.save(file_path)
            result = upload_file_to_s3(bucket, file_path)
        elif operation == 'integrate_lambda_s3_ses':
            bucket = request.form.get('bucket_name')
            key = request.form.get('object_key')
            result = execute_lambda_s3_ses(bucket, key)
        else:
            result = {'error': 'Unsupported operation'}

        return jsonify(result)
    except Exception as error:
        return jsonify({'error': str(error)}), 500


if __name__ == '__main__':
    app.run(port=8000)
